In [1]:
import logging

"""
This class represents the tree data structure.
Each node has a dict value and a list of children.
"""


class Node:
    def __init__(self, key, value=None):
        self.key = key
        self.value = value
        self.children = []

    def add_child(self, child):
        self.children.append(child)

    def __repr__(self, level=0):
        ret = "\t" * level + f"{self.key}: {self.value}\n"
        for child in self.children:
            ret += child.__repr__(level + 1)
        return ret

    def __str__(self):
        return self.__repr__(0)

    @staticmethod
    def parse_json(key, data):
        if isinstance(data, dict):
            node = Node(key)
            for key, value in data.items():
                node.add_child(Node.parse_json(key, value))
            return node
        if isinstance(data, list):
            node = Node(key)
            for i, item in enumerate(data):
                node.add_child(Node.parse_json(f"[{i}]", item))
            return node
        else:
            return Node(key, data)



In [2]:
data = """{
"name":"Homepage",
"type":"FRAME",
"visible":true,
"x":816,
"y":100,
"width":1440,
"height":8406,
"rotation":0,
"opacity":1,
"cornerRadius":0,
"cornerSmoothing":0,
"constraints":
{
	"horizontal": "MIN",
	"vertical": "MIN"
}
}
"""

In [3]:
import json

tree = Node.parse_json("root", json.loads(data))

In [4]:
print(tree)

root: None
	name: Homepage
	type: FRAME
	visible: True
	x: 816
	y: 100
	width: 1440
	height: 8406
	rotation: 0
	opacity: 1
	cornerRadius: 0
	cornerSmoothing: 0
	constraints: None
		horizontal: MIN
		vertical: MIN



In [5]:
json_a = json.loads(
    """
    [
    { "name": "Son", "age": 40},
    { "name": "Huong", "age": 44}
    ]
    """
)

In [6]:
json_a

[{'name': 'Son', 'age': 40}, {'name': 'Huong', 'age': 44}]

In [7]:
type(json_a)

list

In [8]:
tree_a = Node.parse_json("root", json_a)

In [9]:
tree_a

root: None
	[0]: None
		name: Son
		age: 40
	[1]: None
		name: Huong
		age: 44

In [10]:
type(tree_a)

__main__.Node

In [11]:
print(json_a)

[{'name': 'Son', 'age': 40}, {'name': 'Huong', 'age': 44}]


In [12]:
print(tree_a)

root: None
	[0]: None
		name: Son
		age: 40
	[1]: None
		name: Huong
		age: 44



In [13]:
class PFStyle:
    def __init__(self, json_data):
        if "_id" in json_data:
            self.item_id = json_data["_id"]
        elif "id" in json_data:
            self.item_id = json_data["id"]
        else:
            print("Error: no id - data: ", json_data)
            return
        if "type" in json_data:
            self.type = json_data["type"]
        else:
            self.type = None
        if "styles" in json_data:
            self.styles = json_data["styles"]
        else:
            self.styles = {}

    def __repr__(self):
        return json.dumps(self.__dict__, indent=4)

In [14]:
class PFNode:
    def __init__(self, json_data):
        if "_id" in json_data:
            self.id = json_data["_id"]
        elif "id" in json_data:
            self.id = json_data["id"]
        else:
            print("Error: no id - data: ", json_data)
            return
        if "data" in json_data:
            self.data = json_data["data"]
        else:
            self.data = {}
        if "styles" in json_data:
            self.styles = json_data["styles"]
        else:
            self.styles = []
        if "children" in json_data:
            self.child_ids = json_data["children"]
        else:
            self.child_ids = []
        self.type = json_data["type"]
        self.children = []  # to parse later whe

    def add_child(self, child):
        self.children.append(child)

    def add_style(self, style):
        self.styles.append(style)

    def __repr__(self):
        # return the representation of the node in JSON format
        return json.dumps(self.__dict__, indent=4)



In [15]:
json_str = """{
      "children": [
        "abece676-3ba9-4206-b586-27c705d58a09"
      ],
      "_id": "bcabece6-763b-4972-8675-8627c705d58a",
      "type": "Heading2",
      "data": {
        "value": "Need it by Valentine? Select Express Shipping at Checkout!",
        "placeholder": "Need it by Valentine? Select Express Shipping at Checkout!"
      },
      "styles": []
    }"""
json_data = json.loads(json_str)
json_data

{'children': ['abece676-3ba9-4206-b586-27c705d58a09'],
 '_id': 'bcabece6-763b-4972-8675-8627c705d58a',
 'type': 'Heading2',
 'data': {'value': 'Need it by Valentine? Select Express Shipping at Checkout!',
  'placeholder': 'Need it by Valentine? Select Express Shipping at Checkout!'},
 'styles': []}

In [16]:
exp_node = PFNode(json_data)

In [17]:
exp_node

{
    "id": "bcabece6-763b-4972-8675-8627c705d58a",
    "data": {
        "value": "Need it by Valentine? Select Express Shipping at Checkout!",
        "placeholder": "Need it by Valentine? Select Express Shipping at Checkout!"
    },
    "styles": [],
    "child_ids": [
        "abece676-3ba9-4206-b586-27c705d58a09"
    ],
    "type": "Heading2",
    "children": []
}

In [29]:
import logging

# Set up basic configuration for logging
logging.basicConfig(
    level=logging.DEBUG,  # Set the logging level
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Define the log format
    datefmt='%Y-%m-%d %H:%M:%S'  # Define the date format
)

class PFTree:
    def __init__(self, json_data):
        self.items = []
        self.id2node = {}
        self.id2numid = {}
        idx = 0
        for item_data in json_data["items"]:
            node = PFNode(item_data)
            self.items.append(node)
            self.id2node[node.id] = node
            self.id2numid[node.id] = idx
            idx += 1
            
        self.n = len(self.items)
        print("Number of nodes: ", self.n)

        # parse styles
        self.styles = set()
        for style_data in json_data["styles"]:
            style = PFStyle(style_data)
            item_id = style.item_id
            self.styles.add(style)
            if item_id in self.id2node:
                node = self.id2node[item_id]
                node.add_style(style)
                if len(node.styles) > 1:
                    logging.warning("Multiple styles for node: %s", node.id, " - styles: %s", node.styles)
        print("Number of styles: ", len(self.styles))

        self.__root = None
        self._parse_tree()

    def _parse_tree(self):
        count_incoming = [0] * self.n
        processed_node = [False] * self.n
        for node in self.items:
            if processed_node[self.id2numid[node.id]]:
                print("Error: duplicate node id: ", node.id)
                continue
            for child_id in node.child_ids:
                if child_id in self.id2node.keys():
                    node.add_child(self.id2node[child_id])
                    count_incoming[self.id2numid[child_id]] += 1
                    if count_incoming[self.id2numid[child_id]] > 1:
                        print("Error: multiple incoming edges, id: ", child_id)
                        return
                else:
                    logging.warning("Child id not found in items: %s ", child_id)
            processed_node[self.id2numid[node.id]] = True
        cnt_root = 0
        for node in self.items:
            numid = self.id2numid[node.id]
            if count_incoming[numid] == 0:
                self.__root = node
                cnt_root += 1

        print("cnt_root: ", cnt_root)
        if cnt_root == 0:
            print("Error: no root")
            return None
        if cnt_root > 1:
            print("Error: multiple roots")
            self.__root = None
            return None
        return self.__root

    def __repr__(self):
        return json.dumps(self.__dict__, indent=4)

In [30]:
with open("sample_figma_to_pf_data/FullPage/figma-to-pf.json") as f:
    json_data = json.load(f)  # read json from file "data.json"
tree = PFTree(json_data)

2024-08-28 11:02:13 - root - WARNING - Child id not found in items: 08bb4353-b6f8-4f16-b50e-77271bca703a 


Number of nodes:  832
Number of styles:  601
Error: duplicate node id:  3ecfb804-bcab-4ce6-b63b-a97206758627
cnt_root:  1


In [31]:
debug_node = None
for item in tree.items:
    if item.id == "8627c705-d58a-49e1-8da1-04cd1eb64d68":
        debug_node = item
        print("Found node: ", debug_node.id)
        break

debug_node.child_ids
        

Found node:  8627c705-d58a-49e1-8da1-04cd1eb64d68


['27c705d5-8a09-414d-a104-cd1eb64d680c',
 '05d58a09-e14d-4104-8d1e-b64d680c84d2',
 '8a09e14d-a104-4d1e-b64d-680c84d29973']

In [32]:
type(debug_node.child_ids)

list

In [33]:
json_str = """{
  "customJS": "",
  "customCSS": "",
  "pageflyVersion": "4.13.3",
  "type": "page",
  "globalSectionData": [],
  "items": [
    {
      "_id": "root-id",
      "type": "Container",
      "data": {},
      "styles": [],
      "children": [
        "heading-id",
        "icon-id"
      ]
    },
    {
      "_id": "heading-id",
      "type": "Heading2",
      "data": {
        "value": "Need it by Valentine? Select Express Shipping at Checkout!",
        "placeholder": "Need it by Valentine? Select Express Shipping at Checkout!"
      },
      "styles": [],
      "children": []
    },
    {
      "_id": "icon-id",
      "type": "Icon2",
      "data": {},
      "styles": [],
      "children": [
        "nested-container-id"
      ]
    },
    {
      "_id": "nested-container-id",
      "type": "Container",
      "data": {},
      "styles": [],
      "children": [
        "nested-heading-id"
      ]
    },
    {
      "_id": "nested-heading-id",
      "type": "Heading3",
      "data": {
        "value": "Hurry up, limited time offer!",
        "placeholder": "Hurry up, limited time offer!"
      },
      "styles": [],
      "children": []
    }
  ],
  "styles": [
    {
      "_id": "style-id1",
      "type": "Label",
      "styles": {
        "color": "red",
        "font-size": "20px"
      }
    },
    {
        "_id": "style-id2",
        "type": "Text",
        "styles": {
            "color": "blue",
            "font-size": "30px"
        }
    }
  ]
}
"""
PD = PFTree(json.loads(json_str))

Number of nodes:  5
Number of styles:  2
cnt_root:  1


In [34]:
class FigmaNode:
    def __init__(self):
        self.data = {}
        self.children = []

    def add_data(self, key, value):
        self.data[key] = value

    def add_child(self, child):
        self.children.append(child)

    @staticmethod
    def parse_figma_tree(json_dict: dict):
        node = FigmaNode()
        for key, value in json_dict.items():
            if key != "children":
                node.add_data(key, value)
            else:
                for child in value:
                    node.add_child(FigmaNode.parse_figma_tree(child))
        return node

    def count_nodes(self):
        return 1 + sum(child.count_nodes() for child in self.children)

In [35]:
import json

with open("sample_figma_to_pf_data/FullPage/Figma.json") as f:
    figma_json = json.load(f)
root = FigmaNode.parse_figma_tree(figma_json)
print("Number of nodes: ", root.count_nodes())

Number of nodes:  601


In [36]:
with open("sample_figma_to_pf_data/FullPage/figma-to-pf.json") as f:
    json_data = json.load(f)  # read json from file "data.json"
data = PFTree(json_data)

2024-08-28 11:02:17 - root - WARNING - Child id not found in items: 08bb4353-b6f8-4f16-b50e-77271bca703a 


Number of nodes:  832
Number of styles:  601
Error: duplicate node id:  3ecfb804-bcab-4ce6-b63b-a97206758627
cnt_root:  1


In [37]:
file_names = [
    {"pfdata": "sample_figma_to_pf_data/ContentList/figma-to-pf.json",
     "figma": "sample_figma_to_pf_data/ContentList/RawFigma.json"
     },
    {"pfdata": "sample_figma_to_pf_data/CountDown/figma-to-pf.json",
     "figma": "sample_figma_to_pf_data/CountDown/RawFigma.json"
     },
    {"pfdata": "sample_figma_to_pf_data/CustomerForm/figma-to-pf.json",
     "figma": "sample_figma_to_pf_data/CustomerForm/RawFigma.json"
     },
    {"pfdata": "sample_figma_to_pf_data/FullPage/figma-to-pf.json",
     "figma": "sample_figma_to_pf_data/FullPage/Figma.json"
     },
    {"pfdata": "sample_figma_to_pf_data/NormalButtonAndImages/figma-to-pf.json",
     "figma": "sample_figma_to_pf_data/NormalButtonAndImages/RawFigma.json"
     },
    {"pfdata": "sample_figma_to_pf_data/ProductDetail/figma-to-pf.json",
     "figma": "sample_figma_to_pf_data/ProductDetail/RawFigma.json"
     },
    {"pfdata": "sample_figma_to_pf_data/ProductList/figma-to-pf.json",
     "figma": "sample_figma_to_pf_data/ProductList/RawFigma.json"
     }
]

In [38]:
for file_name in file_names:
    print("=== FILE === ",file_name["pfdata"].split("/")[1] )
    print("=pfdata=")
    with open(file_name["pfdata"]) as f:
        json_data = json.load(f)
    data = PFTree(json_data)
    print("=figma=")
    with open(file_name["figma"]) as f:
        figma_json = json.load(f)
    root = FigmaNode.parse_figma_tree(figma_json)
    print("Number of nodes: ", root.count_nodes())

2024-08-28 11:02:20 - root - WARNING - Child id not found in items: 08bb4353-b6f8-4f16-b50e-77271bca703a 


=== FILE ===  ContentList
=pfdata=
Number of nodes:  190
Number of styles:  134
Error: duplicate node id:  a86453fb-c06d-4e65-88cc-c5e942fe0362
cnt_root:  1
=figma=
Number of nodes:  33
=== FILE ===  CountDown
=pfdata=
Number of nodes:  49
Number of styles:  34
Error: duplicate node id:  f4035e67-b987-41ca-b4da-e4a4e2041dbf
cnt_root:  1
=figma=
Number of nodes:  37
=== FILE ===  CustomerForm
=pfdata=
Number of nodes:  38
Number of styles:  24
cnt_root:  1
=figma=
Number of nodes:  24
=== FILE ===  FullPage
=pfdata=
Number of nodes:  832
Number of styles:  601
Error: duplicate node id:  3ecfb804-bcab-4ce6-b63b-a97206758627
cnt_root:  1
=figma=
Number of nodes:  601
=== FILE ===  NormalButtonAndImages
=pfdata=
Number of nodes:  15
Number of styles:  10
Error: duplicate node id:  caa975e2-2191-44c7-8d8b-2978296854b1
cnt_root:  1
=figma=
Number of nodes:  10
=== FILE ===  ProductDetail
=pfdata=
Number of nodes:  74
Number of styles:  54
Error: duplicate node id:  0396f01b-b018-479d-8406-f1

In [ ]:
# This class will be used to compare the two trees
class TreeComparator:
    def __init__(self, pf_tree, figma_tree):
        self.pf_tree = pf_tree
        self.figma_tree = figma_tree

    def compare_trees(self):
        